Reinitialized existing Git repository in /content/DSS_thesis/.git/


In [ ]:
from google.colab import drive

import pandas as pd


papers = pd.read_csv('/content/drive/My Drive/articles.csv',)
papers

Mounted at /content/drive


,Unnamed: 0,url,text
0,0,https://19thnews.org/2021/07/the-covid-delta-v...,The highly contagious Delta variant of COVID-1...
1,1,https://19thnews.org/2020/11/karen-bass-addres...,We’re the only newsroom dedicated to writing a...
2,2,https://19thnews.org/2020/08/kamala-harris-com...,Read the latest story on how LGBTQ+ Americans ...
3,3,https://19thnews.org/2020/11/kim-ng-mlb-first-...,We’re the only newsroom dedicated to writing a...
4,4,https://19thnews.org/2020/12/shirley-sherrod-h...,"As a nonprofit newsroom, members are critical ..."
...,...,...,...
2714,2780,https://www.zerohedge.com/political/state-wash...,"Authored by Jonathan Turley,\n\nThe House Demo..."
2715,2781,https://www.zerohedge.com/geopolitical/biden-k...,The Biden administration has said it will cont...
2716,2793,https://www.zerohedge.com/political/we-want-re...,Update (2031ET): Someone on Twitter points out...
2717,2794,https://www.zerohedge.com/technology/here-are-...,"Over the past week, President Trump has been k..."


In [ ]:
# Remove the columns
papers = papers.drop(columns=['Unnamed: 0', 'url'], axis=1)
# sample only 100 papers
papers = papers.sample(100)
# Print out the first rows of papers
papers.head()

,text
1515,"Recently, the media disclosed that Transportat..."
308,Kamala Harris has not lived up to the hype.\n\...
2101,"Over the weekend, Madison Cawthorn, the Trumpy..."
1009,"Officials at IU Health, the state's largest he..."
2106,In the week following Liz Cheney’s purge for t...


In [ ]:
# Load the regular expression library
import re
# Remove punctuation
papers['text_processed'] = papers['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['text_processed'] = papers['text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['text_processed'].head()

1515    recently the media disclosed that transportati...
308     kamala harris has not lived up to the hype\n\n...
2101    over the weekend madison cawthorn the trumpy y...
1009    officials at iu health the state's largest hea...
2106    in the week following liz cheney’s purge for t...
Name: text_processed, dtype: object

In [ ]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = papers.text_processed.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['recently', 'the', 'media', 'disclosed', 'that', 'transportation', 'secretary', 'pete', 'buttigieg', 'took', 'two', 'months', 'of', 'parental', 'leave', 'following', 'the', 'adoption', 'of', 'twins', 'this', 'news', 'led', 'to', 'arguments', 'online', 'about', 'whether', 'he', 'should']


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
# NLTK Stop words
!pip install nltk
!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

import pandas as pd
import matplotlib.pyplot as plt
import io
import unicodedata
import numpy as np
import re
import string

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['recently', 'medium', 'disclose', 'take', 'month', 'parental', 'leave', 'follow', 'adoption', 'news', 'lead', 'argument', 'online', 'do', 'global', 'supply', 'chain', 'crisis', 'man', 'take', 'parental', 'leave', 'privileged', 'class', 'get', 'much', 'time', 'miss', 'significant', 'opportunity', 'defend', 'absence', 'leave', 'loudly', 'declare', 'importance', 'parent', 'spending', 'time', 'child', 'advocate', 'family', 'spending', 'time', 'together', 'note', 'importance', 'parent', 'child', 'bond', 'agree', 'wholeheartedly', 'point', 'challenge', 'policy', 'push', 'year', 'discourage', 'family', 'formation', 'extend', 'adolescence', 'make', 'housing', 'expensive', 'deemphasize', 'role', 'father', 'time', 'conservative', 'change', 'strategy', 'scream', 'stop', 'stand', 'athwart', 'history', 'say', 'completely', 'different', 'young', 'adult', 'country', 'marry', 'less', 'few', 'child', 'trend', 'feature', 'society', 'decline', 'strong', 'family', 'building', 'block', 'free', 'prosperou

In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 3), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 4), (13, 2), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 2), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 3), (44, 1), (45, 1), (46, 1), (47, 3), (48, 1), (49, 1), (50, 1), (51, 1), (52, 14), (53, 2), (54, 6), (55, 7), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 4), (69, 6), (70, 1), (71, 1), (72, 2), (73, 2), (74, 4), (75, 1), (76, 2), (77, 1), (78, 1), (79, 1), (80, 3), (81, 1), (82, 1), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 3), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 2), (105, 2), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1

In [ ]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.007*"even" + 0.006*"state" + 0.006*"attack" + 0.005*"threat" + '
  '0.005*"political" + 0.004*"terrorist" + 0.004*"far" + 0.004*"dangerous" + '
  '0.004*"also" + 0.003*"evangelical"'),
 (1,
  '0.013*"say" + 0.006*"people" + 0.006*"year" + 0.005*"make" + 0.005*"child" '
  '+ 0.004*"time" + 0.004*"trump" + 0.004*"family" + 0.004*"many" + '
  '0.004*"haitian"'),
 (2,
  '0.010*"say" + 0.009*"receive" + 0.009*"patient" + 0.007*"plasma" + '
  '0.006*"covid" + 0.006*"library" + 0.005*"case" + 0.005*"program" + '
  '0.005*"death" + 0.004*"child"'),
 (3,
  '0.008*"say" + 0.008*"trump" + 0.005*"write" + 0.005*"go" + 0.005*"agent" + '
  '0.005*"state" + 0.005*"would" + 0.005*"also" + 0.004*"election" + '
  '0.004*"vote"'),
 (4,
  '0.009*"police" + 0.007*"percent" + 0.006*"say" + 0.006*"recycle" + '
  '0.005*"support" + 0.005*"also" + 0.005*"get" + 0.005*"people" + '
  '0.005*"power" + 0.005*"movement"'),
 (5,
  '0.010*"say" + 0.007*"victim" + 0.006*"people" + 0.006*"would" + '
  '0.005*

In [ ]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3059719505928523


In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

  0%|          | 0/540 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
100%|██████████| 540/540 [39:29<00:00,  4.39s/it]


In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [ ]:
lda_tuning_posts= pd.read_csv('lda_tuning_results.csv', lineterminator="\n")

In [ ]:
lda_tuning_posts.sort_values(["Coherence"], ascending=[False]).head(10)

,Validation_Set,Topics,Alpha,Beta,Coherence
207,75% Corpus,8,asymmetric,0.61,0.525579
513,100% Corpus,10,0.01,0.9099999999999999,0.508919
453,100% Corpus,8,0.01,0.9099999999999999,0.507798
538,100% Corpus,10,asymmetric,0.9099999999999999,0.492304
533,100% Corpus,10,symmetric,0.9099999999999999,0.489613
478,100% Corpus,8,asymmetric,0.9099999999999999,0.488428
468,100% Corpus,8,0.9099999999999999,0.9099999999999999,0.482334
202,75% Corpus,8,symmetric,0.61,0.479188
463,100% Corpus,8,0.61,0.9099999999999999,0.463443
183,75% Corpus,8,0.01,0.9099999999999999,0.461955


In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [ ]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

import pickle 
import pyLDAvis
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint
import pyLDAvis.gensim_models



# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
# Visualize the topics


In [ ]:

import pyLDAvis.gensim_models



# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.112174  0.033545       1        1  63.042772
7      0.073130 -0.050989       2        1  29.159865
3     -0.004113  0.018963       3        1   4.435111
4     -0.030883 -0.000815       4        1   1.807110
6     -0.036985  0.002970       5        1   0.720430
2     -0.035426 -0.004109       6        1   0.576472
0     -0.039170  0.000485       7        1   0.152973
5     -0.038728 -0.000051       8        1   0.105268, topic_info=           Term        Freq       Total Category  logprob  loglift
373      worker   79.000000   79.000000  Default  30.0000  30.0000
500       trump  120.000000  120.000000  Default  29.0000  29.0000
416    election  139.000000  139.000000  Default  28.0000  28.0000
299         say  407.000000  407.000000  Default  27.0000  27.0000
12         also  130.000000  130.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
1675   military    0.016495   14.828547   Topic8  -7.8360   0.0551
3120      armed    0.012125    7.914213   Topic8  -8.1437   0.3753
2808  affiliate    0.010750    2.713654   Topic8  -8.2641   1.3252
3455   disperse    0.010657    2.701399   Topic8  -8.2727   1.3211
3452   disorder    0.010630    2.704831   Topic8  -8.2753   1.3173

[452 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1943      1  0.667870      accept
1943      2  0.166967      accept
1943      4  0.083484      accept
4647      1  0.536178  acceptance
4647      2  0.536178  acceptance
...     ...       ...         ...
870       2  0.094390       write
870       3  0.056634       write
376       1  0.680480        year
376       2  0.308007        year
376       3  0.007163        year

[916 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 8, 4, 5, 7, 3, 1, 6])

In [ ]:
!pip install sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
lda_tf = LatentDirichletAllocation(n_components=5, random_state=0)    #n_components = can depend



In [ ]:
for i,topic in enumerate(lda_tf.components):
    print(f'Top 10 words for topic #{i}:')
    print([tf_vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

AttributeError: ignored

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

In [ ]:
cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_tf.topics(formatted=False)

fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

AttributeError: ignored

In [ ]:
# Import the wordcloud library
from wordcloud import WordCloud
# Join the different processed titles together.
long_string = ','.join(list(papers['paper_text_processed'].values))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string)
# Visualize the word cloud
wordcloud.to_image()

KeyError: ignored